In [3]:
import pandas as pd
import datetime as dt
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from datetime import datetime

In [4]:
df_original = pd.read_csv("store_1586_q1_2022_q1_2023.csv")
df_original

,S_NO,CUST_ID,VISIT_ID,VISIT_DT,VISIT_TM,SKU_ID,SKU_DESC,DEPT_ID,DEPT_DESC,SUB_DEPT_ID,SUB_DEPT_DESC,CAT_ID,CAT_DESC,BRANDNUM,BRANDDSC,SALES,VOLUME,STORE_NUM
0,1,95006771,9155991,2022-11-30,17:22:00,30137750,FIRESTONE-WALKER DOUBLE MIND HAZE CAN,15,BEER,45,BEER,364,BEER CRAFT,41350,FIRESTONE,89.8,5,1568
1,10,95006771,9083380,2022-11-01,16:29:00,42867900,SERIOUS BEAN DR PEPPER BEANS,15,MEALS/INGREDIENTS,12,READY TO EAT PREPARED FOODS,96,CANNED FLAVORED BEANS,83050,SERIOUS BEAN CO,59.8,5,1568
2,11,95006771,9099744,2022-11-07,13:11:00,3809150,SNAPPLE PEACH TEA,15,SOFT DRINKS,1,NON CARBONATED BEVERAGES,12,READY TO DRINK TEA AND COFFEE,37532,SNAPPLE,39.8,5,1568
3,12,95006771,9099744,2022-11-07,13:11:00,4189168,SIERRA NEVADA HAZY LITTLE THING IPA CAN,15,BEER,45,BEER,364,BEER CRAFT,22156,SIERRA NEVADA,119.6,10,1568
4,13,95006771,9177872,2022-12-06,09:51:00,24235002,BANANAS ORGANIC,16,PRODUCE,42,FRESH PRODUCE,338,ORGANIC FRUITS,286174,OWN BRANDS,43.6,5,1568
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2496713,199998616,364506787,8409649,2022-02-16,18:34:00,2249450,EL GUAPO BAY LEAVES WHOLE,16,PRODUCE,42,FRESH PRODUCE,339,DRIED PRODUCE,38987,EL GUAPO,59.8,5,1568
2496714,199998622,364506787,8409649,2022-02-16,18:34:00,10107100,CH BEEF TOP LOIN NY STP STK THN BNLS,16,MEAT,44,MEAT,352,"BEEF, CHOICE, NO MANUFACTURER BRAND DECLARED",301709,RANDOM WEIGHT/LAC UNASSIGNED,143.4,5,1568
2496715,199998623,364506787,8409649,2022-02-16,18:34:00,10096550,CH BEEF CHUCK STK BNLS,16,MEAT,44,MEAT,352,"BEEF, CHOICE, NO MANUFACTURER BRAND DECLARED",301709,RANDOM WEIGHT/LAC UNASSIGNED,154.6,5,1568
2496716,199998639,364506787,8409649,2022-02-16,18:34:00,3641504,TILLAMOOK ORIG PRM CKY N CRM,15,FROZEN GROCERY,21,ICE CREAM,168,PACKAGED ICE CREAM,36371,TILLAMOOK,79.8,5,1568


In [5]:
def price_elasticity(df_original):
    df_original = df_original[df_original["VOLUME"] > 0]
    df_original["Unit_price"] = df_original["SALES"]/df_original["VOLUME"]
    df_Stock = df_original.groupby(['VISIT_DT','SKU_ID']).agg({'Unit_price':'mean','VOLUME': 'mean' }).reset_index()
    x_pivot = df_Stock.pivot(index= 'VISIT_DT' ,columns='SKU_ID' ,values='Unit_price')
    x_values = pd.DataFrame(x_pivot.to_records())
    y_pivot = df_Stock.pivot( index = 'VISIT_DT',columns='SKU_ID', values='VOLUME')
    y_values = pd.DataFrame(y_pivot.to_records())
    points = []
    results_values = {
        "name": [],
        "price_elasticity": [],
        "price_mean": [],
        "quantity_mean": [],
        "intercept": [],
        "t_score":[],
        "slope": [],
        "coefficient_pvalue" : [],
    }
    #Append x_values with y_values per same product name
    for column in x_values.columns[1:]:
        column_points = []
        for i in range(len(x_values[column])):
            if not np.isnan(x_values[column][i]) and not np.isnan(y_values[column][i]):
                column_points.append((x_values[column][i], y_values[column][i]))
        df = pd.DataFrame(list(column_points), columns= ['x_value', 'y_value'])


        #Linear Regression Model
        import statsmodels.api as sm
        x_value =df['x_value']
        y_value = df['y_value']
        X = sm.add_constant(x_value)
        model = sm.OLS(y_value, X)
        result = model.fit()
        #(Null Hypothesis test) Coefficient with a p value less than 0.05
        if result.f_pvalue < 0.05:

            rsquared = result.rsquared
            coefficient_pvalue = result.f_pvalue
            intercept,slope = result.params
            mean_price = np.mean(x_value)
            mean_quantity = np.mean(y_value)
            tintercept,t_score = result.tvalues

            #Price elasticity Formula
            price_elasticity = (slope)*(mean_price/mean_quantity)
            #price_elasticity = (slope)*(mean_quantity/mean_price)

            #Append results into dictionary for dataframe
            results_values["name"].append(column)
            results_values["price_elasticity"].append(price_elasticity)
            results_values["price_mean"].append(mean_price)
            results_values["quantity_mean"].append(mean_quantity)
            results_values["intercept"].append(intercept)
            results_values['t_score'].append(t_score)
            results_values["slope"].append(slope)
            results_values["coefficient_pvalue"].append(coefficient_pvalue)

    final_df = pd.DataFrame.from_dict(results_values)
    df_elasticity = final_df[['name','price_elasticity','t_score','coefficient_pvalue','slope','price_mean','quantity_mean','intercept']]
    return df_elasticity

In [6]:
df_elasticity=price_elasticity(df_original)
print(df_elasticity)

c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\regression\linear_model.py:1870: RuntimeWarning: invalid value encountered in scalar divide
  return self.mse_model/self.mse_resid
c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\regression\linear_model.py:1870: RuntimeWarning: invalid value encountered in scalar divide
  return self.mse_model/self.mse_resid
c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\regression\linear_model.py:1870: RuntimeWarning: invalid value encountered in scalar divide
  return self.mse_model/self.mse_resid
c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\regression\linear_model.py:1870: RuntimeWarning: invalid value encountered in scalar divide
  return self.mse_model/self.mse_resid
c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\regression\linear_model.py:1870: RuntimeWarning: invalid value enc

           name  price_elasticity    t_score  coefficient_pvalue     slope  \
0         28950         -3.859296  -3.752465            0.000384 -0.369677   
1         32565         -2.017434  -5.012451            0.000001 -0.722283   
2         32594         -1.554637 -87.200000            0.000131 -1.576420   
3        413722          0.657074   2.593114            0.013791  0.117188   
4        417343         -0.451605  -2.356440            0.027771 -0.065944   
...         ...               ...        ...                 ...       ...   
1697  375151449         -0.331501  -2.639819            0.020403 -0.076601   
1698  375151505         -0.285700  -2.270189            0.026277 -0.098521   
1699  375151719         -0.236530  -2.093725            0.045809 -0.322367   
1700  390230479         -0.679658  -4.985417            0.007568 -0.187810   
1701  465838505        -23.300812  -5.398023            0.000434 -2.024194   

      price_mean  quantity_mean   intercept  
0      62.236308 

In [15]:
def divergent_plot(df, values_column, ylabel, xlabel):

    #Divergent plot
    df['ranking'] = df[values_column].rank( ascending = True).astype(int)
    df.sort_values(values_column, ascending =False, inplace = True)
    # plt.figure(figsize = (20,20), dpi = 80)
    # plt.hlines(y = df['ranking'] , xmin = 0, xmax = df[values_column], alpha = 0.5, linewidth = 3)

    # #Add elasticity labels
    # for x, y, tex in zip(df[values_column], df['ranking'] , df[values_column]):
    #     plt.text(x, y, round(tex, 2), horizontalalignment='right' if x < 0 else 'left',
    #              verticalalignment='center', fontdict={'color':'red' if x < 0 else 'green', 'size':10})


    # # Axis and title
    # plt.gca().set(ylabel= ylabel, xlabel= xlabel)
    # plt.yticks(df['ranking'])
    # plt.title(values_column , fontdict={'size':13})
    # plt.grid(linestyle='--', alpha=0.5)
    # plt.show()


    #Adjust Ranking column and print dataframe
    pd.set_option('display.width', 4000)
    cols = list(df.columns)
    cols = [cols[-1]] + cols[:-1]
    df = df[cols]

    df = df.iloc[:,:3]
    df.set_index('ranking', inplace=True)
    display(df)

In [7]:
def active_customer_data(df_original):
    # Remove invalid Customer IDs
    df_original = df_original[df_original["CUST_ID"] > 0]

    # Convert datatypes
    df_original[["VISIT_ID","SKU_ID","SKU_DESC","DEPT_DESC"]] = df_original[["VISIT_ID","SKU_ID","SKU_DESC","DEPT_DESC"]].astype("string")
    df_original["CUST_ID"] = df_original["CUST_ID"].astype(int)

    # Removing null oberservations
    df_original.dropna(inplace=True)

    # Removing canceled orders and invalid quantities
    df_original = df_original[~df_original["VISIT_ID"].str.contains("C", na=False)]
    df_original = df_original[df_original["VOLUME"] > 0]

    # Add Total Price column
    df_original["TotalPrice"] = df_original["SALES"]

    # Add separate Date column
    df_original['Date'] = pd.to_datetime(df_original['VISIT_DT']).dt.date
    # Defining today date as max(InvoiceDate) + 2 days
    today_date = dt.date (2023, 4, 2)
    print(f" Maximum invoice date: {df_original.VISIT_DT.max()} \n Today date: {today_date}")

    df = df_original.copy()
    df['year']=pd.PeriodIndex(df.Date,freq='Y')
    df['quarter'] = pd.PeriodIndex(df.Date, freq='Q')
    df['month']=pd.PeriodIndex(df.Date, freq='M')
    df['week']=pd.PeriodIndex(df.Date, freq='W')
    df['day']=pd.PeriodIndex(df.Date, freq='D')

    df_pivot = df.pivot_table(index='CUST_ID', columns='week', aggfunc={'Date':'nunique'},fill_value=0)
    df_pivot.columns = df_pivot.columns.droplevel(0) #remove date
    df_pivot.columns.name = None #remove week
    for i in df_pivot.index:
        for j in df_pivot.columns:
            if df_pivot.loc[i,j] >= 1:
             break
            else:
             df_pivot.loc[i][j] = -1

    # Replace -1 with NaN
    df_pivot = df_pivot.replace(-1,np.nan)
    perc_25 = np.nanquantile(df_pivot, 0.25, axis=1)
    df_perc_25 = pd.DataFrame()
    df_perc_25["CUST_ID"] = df_pivot.index
    df_perc_25["Perc_25"] = perc_25
    df_daily = df_perc_25[df_perc_25["Perc_25"] >1.0]

    # weekly customer
    df_nondaily = df_perc_25[df_perc_25["Perc_25"] <=1.0]
    df_nondaily.drop(columns = {"Perc_25"}, axis=1, inplace=True)
    df = df[df['CUST_ID'].isin(df_nondaily["CUST_ID"])]
    df_pivot = df.pivot_table(index='CUST_ID', columns='month', aggfunc={'Date':'nunique'},fill_value=0)
    df_pivot.columns = df_pivot.columns.droplevel(0) #remove date
    df_pivot.columns.name = None
    for i in df_pivot.index:
      for j in df_pivot.columns:
        if df_pivot.loc[i,j]>=1:
         break
        else:
         df_pivot.loc[i][j] =-1

    df_pivot=df_pivot.replace(-1,np.nan)
    perc_25=np.nanquantile(df_pivot, 0.25, axis=1)
    df_perc_25=pd.DataFrame()
    df_perc_25["CUST_ID"]=df_pivot.index
    df_perc_25["Perc_25"]=perc_25
    df_weekly = df_perc_25[df_perc_25["Perc_25"] >= 2.0]

    # Monthly customer
    df_nonweekly = df_perc_25[df_perc_25["Perc_25"] < 2.0]
    df_nonweekly.drop(columns = {"Perc_25"}, axis=1, inplace=True)
    df = df[df['CUST_ID'].isin(df_nonweekly["CUST_ID"])]
    df_pivot = df.pivot_table(index='CUST_ID', columns='quarter', aggfunc={'Date':'nunique'},fill_value=0)
    df_pivot.columns = df_pivot.columns.droplevel(0) #remove date
    df_pivot.columns.name = None
    for i in df_pivot.index:
      for j in df_pivot.columns:
        if df_pivot.loc[i,j]>=1:
         break
        else:
          df_pivot.loc[i][j] =-1

    df_pivot=df_pivot.replace(-1,np.nan)
    perc_25=np.nanquantile(df_pivot, 0.25, axis=1)
    df_perc_25=pd.DataFrame()
    df_perc_25["CUST_ID"]=df_pivot.index
    df_perc_25["Perc_25"]=perc_25
    df_monthly = df_perc_25[df_perc_25["Perc_25"] >=2.0]

    # Quaterly customer
    df_nonmonthly = df_perc_25[df_perc_25["Perc_25"] < 2.0]
    df_nonmonthly.drop(columns = {"Perc_25"}, axis=1, inplace=True)
    df = df[df['CUST_ID'].isin(df_nonmonthly["CUST_ID"])]
    df_pivot = df.pivot_table(index='CUST_ID', columns='year', aggfunc={'Date':'nunique'},fill_value=0)
    df_pivot.columns = df_pivot.columns.droplevel(0) #remove date
    df_pivot.columns.name = None
    for i in df_pivot.index:
     for j in df_pivot.columns:
      if df_pivot.loc[i,j]>=1:
       break
     else:
       df_pivot.loc[i][j] =-1

    df_pivot=df_pivot.replace(-1,np.nan)
    perc_25=np.nanquantile(df_pivot, 0.25, axis=1)
    df_perc_25=pd.DataFrame()
    df_perc_25["CUST_ID"]=df_pivot.index
    df_perc_25["Perc_25"]=perc_25
    df_quarterly = df_perc_25[df_perc_25["Perc_25"] >=3.0]

    # Calculating recency, monetary, frequency and tenure metrics
    rfm = df_original.groupby("CUST_ID").agg({"Date":lambda date: (today_date - date.max()).days,
                                        "VISIT_ID": lambda num: num.nunique(),
                                          "TotalPrice": lambda price: price.sum()}) #total price per customer

    #rfm.columns = rfm.columns.droplevel(0)
    rfm.columns = ['Recency', 'Frequency', "Monetary"]
    df_daily.drop(columns = {"Perc_25"}, axis=1, inplace=True)
    rfm_daily=rfm[rfm["Recency"]<30]
    df_daily_active= rfm_daily[rfm_daily.index.isin(df_daily["CUST_ID"])]

    rfm_daily_inactive=rfm[rfm["Recency"]>30]
    df_daily_inactive = rfm_daily_inactive[rfm_daily_inactive.index.isin(df_daily.CUST_ID)]
    
    q2_F=df_daily_active["Frequency"].quantile(0.50)
    q2_M = df_daily_active["Monetary"].quantile(0.50)
    # create a list of our conditions
    conditions = [
        (df_daily_active['Frequency'] <= q2_F) & (df_daily_active['Monetary']<=q2_M),
        (df_daily_active['Frequency'] <= q2_F) & (df_daily_active['Monetary']>q2_M),
        (df_daily_active['Frequency']>q2_F) & (df_daily_active['Monetary']<=q2_M),
        (df_daily_active['Frequency']>q2_F) & (df_daily_active['Monetary']>q2_M)
        ]

    # create a list of the values we want to assign for each condition
    values = ['LFLM', 'LFHM', 'HFLM', 'HFHM']

    # create a new column and use np.select to assign values to it using our lists as arguments
    df_daily_active['Customer_segments'] = np.select(conditions, values)
    df_daily_active_fn=df_daily_active.groupby('Customer_segments').median().rename(columns= {'Recency':"Median Recency","Frequency":"Median Frequency","Monetary":'Median Monetary'}).join(df_daily_active.Customer_segments.value_counts().rename('count'))
    
    df_weekly.drop(columns = {"Perc_25"}, axis=1, inplace=True)
    rfm_weekly_active=rfm[rfm["Recency"]<60]
    df_weekly_active= rfm_weekly_active[rfm_weekly_active.index.isin(df_weekly["CUST_ID"])]

    rfm_weekly_inactive=rfm[rfm["Recency"]>=60]
    df_weekly_inactive = rfm_weekly_inactive[rfm_weekly_inactive.index.isin(df_weekly.CUST_ID)]
    q2_F=df_weekly_active["Frequency"].quantile(0.50)
    q2_M = df_weekly_active["Monetary"].quantile(0.50)
    # create a list of our conditions
    conditions = [
        (df_weekly_active['Frequency'] <= q2_F) & (df_weekly_active['Monetary']<=q2_M),
        (df_weekly_active['Frequency'] <= q2_F) & (df_weekly_active['Monetary']>q2_M),
        (df_weekly_active['Frequency']>q2_F) & (df_weekly_active['Monetary']<=q2_M),
        (df_weekly_active['Frequency']>q2_F) & (df_weekly_active['Monetary']>q2_M)
        ]

    # create a list of the values we want to assign for each condition
    values = ['LFLM', 'LFHM', 'HFLM', 'HFHM']

    # create a new column and use np.select to assign values to it using our lists as arguments
    df_weekly_active['Customer_segments'] = np.select(conditions, values)
    df_weekly_active_fn=df_weekly_active.groupby('Customer_segments').median().rename(columns= {'Recency':"Median Recency","Frequency":"Median Frequency","Monetary":'Median Monetary'}).join(df_weekly_active.Customer_segments.value_counts().rename('count'))

    df_monthly.drop(columns = {"Perc_25"}, axis=1, inplace=True)
    rfm_monthly_active=rfm[rfm["Recency"]<90]
    df_monthly_active= rfm_monthly_active[rfm_monthly_active.index.isin(df_monthly["CUST_ID"])]

    rfm_monthly_inactive=rfm[rfm["Recency"]>=90]
    df_monthly_inactive= rfm_monthly_inactive[rfm_monthly_inactive.index.isin(df_monthly.CUST_ID)]
    q2_F=df_monthly_active["Frequency"].quantile(0.50)
    q2_M = df_monthly_active["Monetary"].quantile(0.50)
    # create a list of our conditions
    conditions = [
        (df_monthly_active['Frequency'] <= q2_F) & (df_monthly_active['Monetary']<=q2_M),
        (df_monthly_active['Frequency'] <= q2_F) & (df_monthly_active['Monetary']>q2_M),
        (df_monthly_active['Frequency']>q2_F) & (df_monthly_active['Monetary']<=q2_M),
        (df_monthly_active['Frequency']>q2_F) & (df_monthly_active['Monetary']>q2_M)
        ]

    # create a list of the values we want to assign for each condition
    values = ['LFLM', 'LFHM', 'HFLM', 'HFHM']

    # create a new column and use np.select to assign values to it using our lists as arguments
    df_monthly_active['Customer_segments'] = np.select(conditions, values)
    df_monthly_active_fn=df_monthly_active.groupby('Customer_segments').median().rename(columns={"Recency":"Median Recency","Frequency":"Median Frequency","Monetary":"Median Monetary"}).join(df_monthly_active.Customer_segments.value_counts().rename('count'))
    
    df_quarterly.drop(columns = {"Perc_25"}, axis=1, inplace=True)
    rfm_quarterly_active=rfm[rfm["Recency"]<180]
    df_quarterly_active = rfm_quarterly_active[rfm_quarterly_active.index.isin(df_quarterly["CUST_ID"])]

    rfm_quarterly_inactive=rfm[rfm["Recency"]>=180]
    df_quarterly_inactive = rfm_quarterly_inactive[rfm_quarterly_inactive.index.isin(df_quarterly["CUST_ID"])]

    q2_F=df_quarterly_active["Frequency"].quantile(0.50)
    q2_M = df_quarterly_active["Monetary"].quantile(0.50)
    # create a list of our conditions
    conditions = [
        (df_quarterly_active['Frequency'] <= q2_F) & (df_quarterly_active['Monetary']<=q2_M),
        (df_quarterly_active['Frequency'] <= q2_F) & (df_quarterly_active['Monetary']>q2_M),
        (df_quarterly_active['Frequency']>q2_F) & (df_quarterly_active['Monetary']<=q2_M),
        (df_quarterly_active['Frequency']>q2_F) & (df_quarterly_active['Monetary']>q2_M)
        ]

    # create a list of the values we want to assign for each condition
    values = ['LFLM', 'LFHM', 'HFLM', 'HFHM']

    # create a new column and use np.select to assign values to it using our lists as arguments
    df_quarterly_active['Customer_segments'] = np.select(conditions, values)

    df_quarterly_active_fn=df_quarterly_active.groupby('Customer_segments').median().rename(columns={"Recency":"Median Recency","Frequency":"Median Frequency","Monetary":"Median Monetary"}).join(df_quarterly_active.Customer_segments.value_counts().rename('count'))

    return df_daily,df_weekly,df_monthly,df_quarterly,df_daily_active,df_daily_inactive,df_daily_active_fn,df_weekly_active,df_weekly_inactive,df_weekly_active_fn,df_monthly_active,df_monthly_inactive,df_monthly_active_fn,df_quarterly_active,df_quarterly_inactive,df_quarterly_active_fn


In [8]:
df_daily,df_weekly,df_monthly,df_quarterly,df_daily_active,df_daily_inactive,df_daily_active_fn,df_weekly_active,df_weekly_inactive,df_weekly_active_fn,df_monthly_active,df_monthly_inactive,df_monthly_active_fn,df_quarterly_active,df_quarterly_inactive,df_quarterly_active_fn= active_customer_data(df_original)
print(df_weekly_active_fn)
print(df_quarterly_active_fn)


 Maximum invoice date: 2023-03-31 
 Today date: 2023-04-02


C:\Users\hp\AppData\Local\Temp\ipykernel_3372\534445704.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nondaily.drop(columns = {"Perc_25"}, axis=1, inplace=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_3372\534445704.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nonweekly.drop(columns = {"Perc_25"}, axis=1, inplace=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_3372\534445704.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

                   Median Recency  Median Frequency  Median Monetary  count
Customer_segments                                                          
HFHM                          5.0             115.0          97962.2    655
HFLM                          5.0              90.0          41045.0    234
LFHM                          7.0              57.0          77594.7    248
LFLM                          8.0              50.0          29604.2    669
                   Median Recency  Median Frequency  Median Monetary  count
Customer_segments                                                          
HFHM                         27.5              18.0          15876.9    256
HFLM                         26.0              16.0           5852.6    105
LFHM                         23.0              11.0          12326.6    167
LFLM                         23.0              10.0           4774.6    319
